In [1]:
#coding:utf-8
import requests
from bs4 import BeautifulSoup
import bs4
import re
import random
from lxml import etree # 解析数据
from tqdm import tqdm
import pandas as pd

In [64]:
def ua():
    """随机获取一个浏览器用户信息"""
    user_agents = [
        'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
        'Opera/9.25 (Windows NT 5.1; U; en)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
        'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
        'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
        'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
        'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
        'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',
    ]
    agent = random.choice(user_agents)
    return {
        'User-Agent': agent
    }

def get_str(data):
    """格式化输出"""
    return data.strip().replace('问</i>','').replace('\t','').replace('\n','').replace('<i class="ask-tag answer-tag">答</i>','')

def get(url):
    res = requests.get(url,headers=ua())
    return res.text

def main(start_url,start_pg,end_pg):
    question = []
    answer = []
    for i in tqdm(range(start_pg,end_pg)):
        url = start_url + '?page=' + str(i)
        res_text = get(url)
        re_q = re.compile(r'(?<=<i class="ask-tag">)[\s\S]*?(?=</a>)')
        re_a = re.compile(r'(?<=<div class="qa-item qa-item-answer">)[\s\S]*?(?=</div>)')
        result_q=re_q.findall(res_text)
        result_a=re_a.findall(res_text)
        for a in result_q:
            question.append(get_str(a))
        for b in result_a:
            answer.append(get_str(b))
    return question,answer 

In [65]:
start_url = 'https://www.chunyuyisheng.com/pc/qalist/'
question,answer = main(start_url,1,100)

  0%|          | 0/99 [00:00<?, ?it/s]


ProxyError: HTTPSConnectionPool(host='www.chunyuyisheng.com', port=443): Max retries exceeded with url: /pc/qalist/?page=1 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb82a505a60>: Failed to establish a new connection: [Errno 61] Connection refused')))

In [77]:
class spider_qadata:
    def __init__(self,start_url):
        self.symptom_qwds = ['症状', '表征', '现象', '症候', '表现', '不良反应','征兆']
        self.cause_qwds = ['原因', '成因', '为什么', '怎么会', '怎样才', '咋样才', '怎样会', '如何会', '为啥', '为何',
                        '如何才会', '怎么才会', '会导致', '会造成']
        self.acompany_qwds = ['并发症', '并发', '一起发生', '一并发生', '一起出现', '一并出现', '一同发生', '一同出现',
                            '伴随发生', '伴随', '共现']
        self.not_food_qwds = ['不能吃什么','忌口','不能吃什么食物','不能吃什么东西','忌食','少吃']
        self.can_food_qwds = ['吃什么食物','怎么补','吃什么菜','保健品','食谱','宜食','多吃']
        self.drug_qwds = ['药', '药品', '用药', '胶囊', '口服液', '炎片']
        self.check_qwds = ['检查', '检查项目', '查出', '检查', '测出', '试出']
        self.prevent_qwds = ['预防', '防范', '抵制', '抵御', '防止', '躲避', '逃避', '避开', '免得', '逃开', '避开',
                            '避掉', '躲开', '躲掉', '绕开',
                            '怎样才能不', '怎么才能不', '咋样才能不', '咋才能不', '如何才能不',
                            '怎样才不', '怎么才不', '咋样才不', '咋才不', '如何才不',
                            '怎样才可以不', '怎么才可以不', '咋样才可以不', '咋才可以不', '如何可以不',
                            '怎样才可不', '怎么才可不', '咋样才可不', '咋才可不', '如何可不']
        self.lasttime_qwds = ['周期', '多久', '多长时间', '多少时间', '几天', '几年', '多少天', '多少小时', '几个小时',
                            '多少年']
        self.cureway_qwds = ['怎么治疗', '如何医治', '怎么医治', '怎么治', '怎么医', '如何治', '医治方式', '疗法',
                            '咋治', '怎么办', '咋办', '咋治']
        self.cureprob_qwds = ['多大概率能治好', '多大几率能治好', '治好希望大么', '几率', '几成', '比例', '可能性',
                            '能治', '可治', '可以治', '可以医']
        self.easyget_qwds = ['易感人群', '容易感染', '易发人群', '什么人', '哪些人', '感染', '染上', '得上']
        self.desc_qwds = ['是怎么样的','']
        self.start_url = start_url
    def ua(self):
        """随机获取一个浏览器用户信息"""
        user_agents = [
            'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
            'Opera/9.25 (Windows NT 5.1; U; en)',
            'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
            'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
            'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
            'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
            'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
            'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',
        ]
        agent = random.choice(user_agents)
        return {
            'User-Agent': agent
        }
    
    def get_str(self,data):
        """格式化输出"""
        return data.strip().replace('问</i>','').replace('\t','').replace('\n','').replace('<i class="ask-tag answer-tag">答</i>','').replace('<span class=\'s-hl\'>','').replace('</span>','')
    
    def get(self,url):
        """解析网页"""
        res = requests.get(url,headers=self.ua())
        return res.text
    
    def spider(self,start_pg,end_pg,qa):
        question = []
        answer = []
        if qa == 'disease_symptom':
            qatype = self.symptom_qwds
        elif qa == 'disease_cause':
            qatype = self.cause_qwds
        elif qa == 'disease_acompany':
            qatype = self.acompany_qwds
        elif qa == 'disease_not_food':
            qatype = self.not_food_qwds
        elif qa == 'disease_do_food':
            qatype = self.can_food_qwds
        elif qa == 'disease_drug':
            qatype = self.drug_qwds
        elif qa == 'disease_check':
            qatype = self.check_qwds
        elif qa == 'disease_prevent':
            qatype = self.prevent_qwds
        elif qa == 'disease_lasttime':
            qatype = self.lasttime_qwds
        elif qa == 'disease_cureway':
            qatype = self.cureway_qwds
        elif qa == 'disease_cureprob':
            qatype = self.cureprob_qwds
        elif qa == 'disease_easyget':
            qatype = self.easyget_qwds
        for i in qatype:
            for j in range(start_pg,end_pg):
                url = self.start_url + '?query=' + str(i) + '&page=' + str (j)
                res_text = self.get(url)
                re_q = re.compile(r'(?<=<i class="ask-tag">)[\s\S]*?(?=</a>)')
                re_a = re.compile(r'(?<=<div class="qa-item qa-item-answer">)[\s\S]*?(?=</div>)')
                result_q=re_q.findall(res_text)
                result_a=re_a.findall(res_text)
                for a in result_q:
                    question.append(self.get_str(a))
                for b in result_a:
                    answer.append(self.get_str(b))
        return question,answer 

In [78]:
data = pd.read_csv('/Users/zhouyang/Desktop/problem_label.txt', delimiter=' ', header=None, names=['意图类型', '编号', '中文名', '重新编号'])

In [79]:
spider_qa = spider_qadata('https://www.chunyuyisheng.com/pc/search/qalist/')
qa_data = pd.DataFrame(columns=['question', 'answer', 'question_type'])
a = 0
for i in tqdm(data['意图类型']):
    question,answer = spider_qa.spider(1,10,i)
    question_df = pd.DataFrame(question,columns=['question'])
    answer_df = pd.DataFrame(answer,columns=['answer'])
    qa_type = pd.concat([question_df,answer_df],axis=1)
    qa_type['question_type'] = data.loc[a,'重新编号']
    a += 1
    qa_data = pd.concat([qa_data,qa_type], axis=0)

100%|██████████| 12/12 [06:58<00:00, 34.91s/it]


In [80]:
qa_data.to_csv('问答对数据集.csv')

In [81]:
qa_data_dp = qa_data.drop_duplicates(subset=['question','answer','question_type'], keep = 'first')

In [82]:
qa_data_dp.to_csv('问答对数据集.csv')

In [86]:
qa_data_dpa = qa_data_dp.drop_duplicates(subset=['question','answer'], keep=False)

In [87]:
qa_data_dpa.to_csv('问答对数据集.csv')

In [84]:
qa_data_dp[qa_data_dp.duplicated(subset=['question','answer'],keep=False)]

,question,answer,question_type
5,"酒瘾严重,不喝酒会手抖等症状",你好，这种情况多久了呢？现在每天大概怎样的喝酒量除了手抖，还有其它症状没有你现在想咨询什么问...,1
20,飞蚊症用药物治疗到底能不能消除症状,你好，只能减轻，彻底消除不了的石斛夜光丸需要长期吃不同牌子价钱不一样，但是不贵您说的这个主要...,1
31,白带乳白色伴有瘙痒症状，还怎样用药...,你好，是豆腐渣样的吗最好查个白带常规再针对性用药不能随便用药的，要查白带看是哪种类型的阴道炎...,1
49,近期感觉头晕目眩，伴随呕吐症状（女...,这种情况有多长时间了？每天都这样？有没有耳鸣，听力下降？做过头颈磁共振和血管的检查吗这个都会...,1
142,感冒10天后出现耳鸣现象怎么回事,感音神经性耳聋。原来听力好么？属于突发性耳聋需要及时输液治疗你要是想听力好一些，建议输液，好...,1
...,...,...,...
1446,"库欣综合征好医不,要手术吗",您好，非常乐意为您解答问题这个病，如果找到肾上腺有占位了，最好的办法是手术硝苯地平缓释片您现...,11
1447,"出现宫寒,怎么办怎么医",你好，请问有什么症状？宫寒，湿气大，平时饮食不要吃寒凉食物。可以艾灸服用艾附暖宫丸晚上经常用...,11
1448,脑鸣好医吗?,你好，你这个症状有多长时间了？还伴有其它什么不适表现吗是脑鸣还是耳鸣？既往有无脑部外伤或脑血...,11
1449,医比克小孩口腔能用吗,你好！请问宝宝有什么症状？多长时间了？恢复的怎么样？不要自己给宝宝用药，手术后用药要咨询医生...,11


In [14]:
a = requests.get('https://www.chunyuyisheng.com/pc/search/qalist/?query=%E7%97%87%E7%8A%B6')
html = etree.HTML(a.text)
question = []
answer = []
q= html.xpath('//div[@class = "qa-item qa-item-ask"]//a/text()')
a= html.xpath('//div[@class = "qa-item qa-item-answer"]/text()')
q = [item for item in q if item != '\n\t\t\t\t\t']
a = [item for item in a if item != '\n\t\t\t\t']
for x,y in zip(q,a):
    question.append(x)
    answer.append(y)
answer

['\n\t\t\t\t末次月经是什么时候？有出血或者腹痛的情况吗你这个不太好哎看起来像是宫外孕怀疑左侧输卵管妊娠多半是因为输卵管有炎症，通畅度不好明天换家医院，复查一下如果今晚出现突然腹痛加重的情况，要立刻线下就诊高度怀疑，毕竟B超没看到孕囊只是报了增粗阴超准是的建议阴超有可能的巧囊是内膜异位到卵巢造成的如果确诊宫外孕，需要手术治疗的话，可以一起解决血值低或者发育慢的话可以考虑保守治疗保守治疗也要住院进行以防突发情况现在一般都腹腔镜受当地医疗条件限制还有你的病情发展情况杀胚药不要擅自用药！！！要在医生指导下用药是的！不要犯傻，宫外孕大出血是有生命危险的疑似输卵管还没看到云南输卵管还没看到孕囊\n\t\t\t',
 '\n\t\t\t\t你好！这种情况有多长时间了？天旋地转吗你晕的时候看东西转吗？有没有耳鸣？平时有什么基础疾病吗嗯嗯，我认为你要查一个核磁先排除脑袋里问题比如长了东西，血管狭窄等嗯嗯嗯嗯\n\t\t\t',
 '\n\t\t\t\t您好，请问宝宝鼻涕流的很多吗吃点抗感颗粒吧这次应该是普通感冒先吃点抗感颗粒就可以注意多喝水喷鼻对鼻塞效果还可以，流鼻涕效果不太好最好是吃点药中成药，副作用不大的，也不是很苦治疗感冒的家里有什么药，可以发给我看看有没有可以吃的豉翘可以，其他不咳嗽不建议吃，氨酚是复方制剂，也不建议吃目前不需要去医院有可能倒流的如果不放心可以去医院看看的，但是药肯定是要吃点的可以的小儿氨酚是复方制剂，副作用大，不建议使用也可以只要不再加重就没事可以的红霉素或头孢都可以可以的\n\t\t\t',
 '\n\t\t\t\t肚子疼，呕吐，呃逆，食欲差，每个人不一定全都出现有泡泡查个尿常规，看里面有没有蛋白这和胃没关系那就不用管它你发的慢性胃炎很难肌酐高才是肾炎，你是低啊肾炎一般到后期肾功能很差的时候才会发现，基本上就该透析了，前期没什么',
 '。所以需要定期查尿常规蛋白，隐血，红细胞，说明不是尿常规都查了没事，你纠结这干啥？不能，彩超看的是肾脏结构有没有问题，尿常规看的是肾脏功能。综合看两个都要看才能诊断肾炎没事，放心的睡吧\n\t\t\t',
 '\n\t\t\t\t你好，请问这种情况多久了有无图片能否发下包皮环的试样不建议用包皮环，建议每天清洁卫生即可因为之前包皮包住龟头，现在用包皮环，龟头表面干燥，会有起皮不会，可以用维生素E涂抹表面没有感染要

In [1]:
symptom_qwds = ['症状', '表征', '现象', '症候', '表现', '不良反应','征兆']
cause_qwds = ['原因', '成因', '为什么', '怎么会', '怎样才', '咋样才', '怎样会', '如何会', '为啥', '为何',
                '如何才会', '怎么才会', '会导致', '会造成']
acompany_qwds = ['并发症', '并发', '一起发生', '一并发生', '一起出现', '一并出现', '一同发生', '一同出现',
                    '伴随发生', '伴随', '共现']
not_food_qwds = ['不能吃什么','忌口','不能吃什么食物','不能吃什么东西','忌食','少吃']
can_food_qwds = ['吃什么食物','怎么补','吃什么菜','保健品','食谱','宜食','多吃']
drug_qwds = ['药', '药品', '用药', '胶囊', '口服液', '炎片']
check_qwds = ['检查', '检查项目', '查出', '检查', '测出', '试出']
prevent_qwds = ['预防', '防范', '抵制', '抵御', '防止', '躲避', '逃避', '避开', '免得', '逃开', '避开',
                    '避掉', '躲开', '躲掉', '绕开',
                    '怎样才能不', '怎么才能不', '咋样才能不', '咋才能不', '如何才能不',
                    '怎样才不', '怎么才不', '咋样才不', '咋才不', '如何才不',
                    '怎样才可以不', '怎么才可以不', '咋样才可以不', '咋才可以不', '如何可以不',
                    '怎样才可不', '怎么才可不', '咋样才可不', '咋才可不', '如何可不']
lasttime_qwds = ['周期', '多久', '多长时间', '多少时间', '几天', '几年', '多少天', '多少小时', '几个小时',
                    '多少年']
cureway_qwds = ['怎么治疗', '如何医治', '怎么医治', '怎么治', '怎么医', '如何治', '医治方式', '疗法',
                    '咋治', '怎么办', '咋办', '咋治']
cureprob_qwds = ['多大概率能治好', '多大几率能治好', '治好希望大么', '几率', '几成', '比例', '可能性',
                    '能治', '可治', '可以治', '可以医']
easyget_qwds = ['易感人群', '容易感染', '易发人群', '什么人', '哪些人', '感染', '染上', '得上']
question_type = [symptom_qwds, cause_qwds, acompany_qwds, not_food_qwds, can_food_qwds, drug_qwds, check_qwds, prevent_qwds, lasttime_qwds, cureway_qwds, cureprob_qwds, easyget_qwds]
s = 0
for i in question_type:
    s += len(i)

In [2]:
s

133